# Lab 2: Naive Bayes

In this lab we'll be training and evaluating a Naive Bayes's classifier on our two triangles datasets to predict whether a participant is depressed/scizophrenic. 

First off, let's start with some imports, as usual.

In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

Now, load the datasets from Lab1 and save them as ```depression_data``` and ```scizophreania_data```. Display the first 5 rows of each dataset, to see if everything looks ok.

In [22]:
# Code here.
data_path_s = os.path.join(os.getcwd(), '../data', 'triangles_schizophrenia.csv')
data_s = pd.read_csv(data_path_s,index_col=0)
data_path_d = os.path.join(os.getcwd(), '../data', 'triangles_depression.csv')
data_d = pd.read_csv(data_path_d,index_col=0)
data_s.head()

,de,der,trekanter,sdan,flyver,rundt,i,den,firkant,frem,...,killede,forstod,forskrkkelse,snuser,mh,nja,fatter,besg,pile,label
0,5.0,5.0,2.0,0.0,0.0,4.0,5.0,27.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Controls
1,7.0,0.0,1.0,1.0,0.0,2.0,4.0,16.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Controls
2,3.0,6.0,6.0,0.0,0.0,2.0,8.0,16.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Controls
3,7.0,12.0,0.0,0.0,0.0,7.0,18.0,40.0,11.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Controls
4,1.0,5.0,1.0,1.0,0.0,1.0,2.0,14.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Controls


In [4]:
data_d.head()

,det,er,simpelthen,at,den,lille,laver,noget,og,s,...,rolige,opfrer,genert,pgende,smforelsket,oplevede,flakkede,fornemmelsen,besgte,label
0,4.0,2.0,0.0,2.0,18.0,0.0,0.0,1.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dc
1,4.0,2.0,0.0,5.0,6.0,0.0,0.0,5.0,6.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dc
2,12.0,10.0,0.0,9.0,12.0,6.0,0.0,6.0,16.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dc
3,8.0,2.0,0.0,0.0,3.0,1.0,0.0,3.0,8.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dc
4,5.0,4.0,0.0,0.0,18.0,2.0,0.0,2.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dc


## 1. sklearn and the object-oriented framework

Before we get started, now is maybe a good time for a quick introduction to object-oriented programming, and how it is applied in the ```sklearn``` library. ```scikit-learn```, or ```sklearn``` for short is a machine learning toolkit containing many useful things, ranging from classifiers such as Naive Bayes or Logistic Regression, over data transformation techniques such as PCA/dimensionality reduction, to preprocessing and evaluation modules. Stop here and take a browse through its <a href="http://scikit-learn.org/stable/documentation.html">documentation</a>, to get an overview. You do not need to understand everything, or study it front to back, but try to get a feel for what kinds of stuff it contains, and how the programs using it are typically structured.

Like many python libraries (one you might be quite familiat with is ```psychopy```), sklearn is based on object-oriented programming. Let's start with a quick intro, since I think understanding objects, methods and attributes will help you understand what's going on with the code we'll be using below. Personally, it took me quite some time to figure out exactly what objects are, and why they are so useful. If you have done any programming course in your life, you will have come across them sooner or later, and you'll probably have seen some pretty stupid examples, like this one:

In [1]:
#let's make a bike:
class Bike:
    def __init__(self,height,wheel_diameter):
        #it has properties, such as a height and a wheel diameter. it's position always starts at 0
        self.height = height
        self.wheel_diameter = wheel_diameter
        self.position = 0
        
    def cycle(self,speed):
        #when you cycle, the bike moves forward by the speed times the wheel diameter 
        #(this is not real cycling, I have no idea about the physics of it)
        self.position = self.wheel_diameter * speed
    
    def get_position(self):
        #the user can call this function to locate the bike at its current position
        return self.position

#okay. let's ride the bike. first, we *instantiate* the object:
my_supercool_bike = Bike(120,70) #let's just take some numbers (that could be centimeters) for the height and wheel

#let's find out the bike's initial position:
initial_pos=my_supercool_bike.get_position()
print("Initial position of the bike: {}".format(initial_pos))

#let's ride the bike
my_supercool_bike.cycle(10) #let's say we cycle at 10km/h

#check the position again
final_pos=my_supercool_bike.get_position()
print("Final position of the bike: {}".format(final_pos))

Initial position of the bike: 0
Final position of the bike: 700


Easy. Everything clear?! 

Of course not. While this shows the basic structure of defining your own object, it doesn't really tell you why objects are useful, or what they are, really.

Let's start with what the example <b>does</b> show: 
<ul>
<li>Objects are defined with the ```class``` statement
<li>They have <b>methods</b> and <b>attributes</b>. <b>Methods</b> are like functions, and you can call them with ```object.method()```, e.g. ```Bike.cycle()```. <b>Attributes</b> are internal variables that the object stores.
<li>Within the object, the word ```self``` is placed instead of the object name. When the object is instantiated, the variable that is used to instantiate the object replaces ```self``` on the outside. For example, if the method ```cycle``` were to be called inside ```get_position```, we'd use ```self.cycle()```, but once we have our own <b>instance</b> of the object (my_supercool_bike), we call the method by using the object name. The same goes for attributes of the object. 
<li>Every object has an ```__init__``` method that automatically gets called whenever a new object of this type is instantiated. Don't worry about the details of this now. In the case of our bike, this just means that we fill in the attribute values specified by the user, and start at a specific position.
</ul>

What the example fails to show, is why on earth anyone would find this useful. It's just as easy to get the same result using regular variables and functions, like so:

In [2]:
#define some initial variables
my_wheel_diameter = 70
my_position = 0
my_speed=10
        
def cycle(speed,wheel_diameter,position):
    position = wheel_diameter * speed
    return position
    
#let's find out the bike's initial position:
initial_pos=my_position
print("Initial position of the bike: {}".format(initial_pos))

#cycle and check the position again
final_pos=cycle(my_speed,my_wheel_diameter,my_position)
print("Final position of the bike: {}".format(final_pos))

Initial position of the bike: 0
Final position of the bike: 700


So, why bother? The reason why classes are useful is exactly their ability to store attributes and methods <b>internally</b>. In the above code, we had to define some global variables in the beginning of our script, and pass them into the function(s) we want to use. Whatever value is returned has to be stored in another variable, or else is forever lost. The function does not remember anything. Objects <b>remember</b> their attribute values. If you want to cycle again, the ```Bike``` object above will remember its current position and wheel diameter, and take it from there. The standalone cycle function won't do that - you have to pass its current position as an argument.

Imagine cooking an elaborate meal, but you only have one pan (a function) and you need to fill anything you make into different plates from your cupboard (variables). Now imagine instead you have this super fancy kitchenaid, which automatically stores all the intermediate results and procedures in some secret hidden place (nobody cares where that is), but you can ask it for a cake or the recipe, anytime you want.

For Machine Learning, this ability of objects to store and remember (so to say: "learn") information is extremely useful. For example, you can instantiate a specific type of model, train its parameters on some data, and then run it on that same data, additional data, etc. You might even throw away all the data altogether, and just keep the model parameters. All you need is the model object, and it remembers where its parameters (e.g., probabilities of observing certain words) are currently at. For some models, you can even update some parameters, using only new, freshly acquired data.

Don't worry if the above introduction is not entirely clear to you. Hopefully you start thinking about python and the world of objects, and it helps you make sense about the way models and methods are used with sklearn. Btw, you probably already know quite a few objects: python strings, lists and dictionaries are all their own type of object, and they have class-specific methods you can call, such as ```.lower()``` for a string, or ```.sort()``` for a list. Psychopy modules also consist largely of objects.

## 2. Train-test split 

Let's start with the schizophrenia dataset. We'll build a model which predicts, for each instance/participant, whether they are schizophrenic or not.

First, we need to split out dataset into a training and test set. We don't have enough data for a proper validation set; we can use cross-validation if we were to tune some hyperparameters. For now, let's keep things simple and use the default value for add-alpha smoothing.

Before you start coding, think about a sensible way to divide the data. How much can we afford to reserve for testing? How much test data do we need? Are the classes very imbalanced? Should we choose our test set randomly, or make sure the distribution of classes is the same for the trainset and testset?

*Take some notes here:*

First, we want to split the data into features and labels seperately, and save each of them in a numpy array. This is helpful for some of the sklearn function we want to use later. As a convention in Machine learning, we use upper case letters (X) to represent matrices and lower case letters (y) for vectors:

In [25]:
#split the data into features and labels, save them each as numpy.arrays:
X = data_s.drop("label",axis=1).values #X is our feature matrix
y = data_s["label"].values #y is our vector of labels

Sklearn conveniently splits our dataset for us: Use the <a href="http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">```train_test_split()```</a> function. Save the output of that function in 4 different variables: ```X_train, X_test, y_train, y_test```. Set the percentage of test instances to what you have decided above (it is entered as a float between 0 and 1, 1 being 100%). Our train/test set is pretty small, and therefore vulnerable to random sampling. Therefore, set ```stratify=y```. This ensures the distribution of classes will be about the same in training and testing (is this a realistic assumtion?). Set the ```random_state``` argument of that function =42.

Test the output by using the ```.shape``` attribute of the resulting test and training features/arrays. How many are there in each dataset? Is the feature dimension what you'd expect?

*Note: Many sklearn functions take a ```random_state```. Why do you have to set it? What is it for? Where is there randomness in this function, and where do you think could there be random elements in other Machine Learning classifiers and tools?*

In [26]:
#Code here.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

*Take notes here.*

## 3. A dummy baseline

To see how good our classifier can become, we first need to establish a baseline. In actual research, the baseline is usually the current state-of-the-art. Just to see if we can get anything done at all, we usually also use what is called a <b>dummy baseline</b>. This is the worst performance we can get, without training our model. 

One way of establishing a dummy baseline is to choose a random class for each instance, and measure the accuracy. However, imagine in our dataset, 90% of all participants are actually not scizophrenic, and only 10% are. A completely random classifier would do much worse than one that predicts that every person is not scizophrenic. However, as a diagnosis tool, those two dummy classifiers are equally useless.

Our second dummy classifier therefore selects the most frequent class in the training set for every test person. If this were to give 90% accuracy (because 90% of all participants are in fact in the most frequent class), then we'd hope for our actual model (the Naive Bayes classifier) to get over 90% correct. Failing to beat the dummy baseline can hint at potential problems with our model: Maybe we have too little data, too many features, false assumptions, or just a buggy implementation.

Below, establish two dummy baselines:

1. A random one (using np.random.rand() to create an output array of the size of the dataset (number of participants))
2. One that chooses the most frequent class in the training set for each test instance (you should be able to figure this one out yourself; hint: determine the most frequent class by looking at the training labels, the measure how many participants fall into that class in the test set)

In [ ]:
#Code here.


## 4. The Multinomial Naive Bayes Classifier

Let's start building a basic model. We'll use the <a href="http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB">```MultinomialNB```</a> class, since our output variable is multinomial (technically binomial but we'll ignore that for now). Have a look at the documentation to see what arguments it takes, and how the model is trained.

Afterwards:
- Instantiate the model, using the default parameters (for now)
- Fit the model on the training data, using its ```.fit()``` method
- Predict the test labels using the test features, and save them in an array called ```y_pred``` (for predicted)
- Print the first 5 predicted labels and the first 5 true labels (```y_test```) above one another and compare them. Does the classifier seem to be working?

In [27]:
# Code here.
nb = MultinomialNB()
nb.fit(X_train,y_train)
y_pred = nb.predict(X_test)
print(y_pred[:5])
print(y_test[:5])

['Controls' 'Patients' 'Patients' 'Controls' 'Patients']
['Patients' 'Controls' 'Patients' 'Controls' 'Patients']


Doesn't look too convincing so far. Let's do some proper evaluation to find out how well we are doing. You can use the <a href="http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html">```accuracy_score```</a> function from ```sklearn.metrics``` to get the accuracy of the classifier. (Alternatively, nb also has a ```.score()``` method.)
Compare accuracy against your dummy baslines. Can we beat that? Reflect on possible causes for poor performance. 

Additionally, you may want to display precision, recall and F1, to see what kinds of errors our classifier most commonly makes. (*Hint: sklearn.metrics provides you with the function <a href="http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html">```precision_recall_fscore_support```</a>*)

In [28]:
# Code here.
print("accuracy: {}".format(accuracy_score(y_test,y_pred)))
precision, recall, f1, support = precision_recall_fscore_support(y_test,y_pred)
print("precision (class1): {}, precision (class2): {},\nrecall (class1) : {}, recall (class2): {}".format(precision[0],precision[1],recall[0],recall[1]))

accuracy: 0.5806451612903226
precision (class1): 0.5714285714285714, precision (class2): 0.5882352941176471,
recall (class1) : 0.5333333333333333, recall (class2): 0.625


*Take Notes here.*

Examine the class priors out model has learned (*Hint: chech out the documentation for the <a href="http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB">```MultinomialNB```</a> class to find which attribute to look at*). Does the result match your expectations (*you may have to convert log probabilities into regular ones with ```np.exp()``` to make sense of it*)?

In [30]:
#Code here.
np.exp(nb.class_log_prior_)

array([0.5, 0.5])

Imagine we had collected a dataset with 50% schizophrenic people, but the fraction of people with the condition is much smaller in the actual test population. Let's say, only 10% of all people tested with this classifier (for whom we record some data describing triangles, to find out whether they are schizophrenic) are actually schizophrenic. We know this because of general statistics or the experience of a doctor. Do we have to change our classifier? Can you find out which attribute you would have to change in the MultinomialNB model?

*Take notes here:*

Let's examine the log probabilities the model has learned for words being used by patients versus controls. 
- Which attribute will display the individual feature (word) log probabilities? 
- Find the positions where the log probabilities are highest (> -4.5 in log space, or ca. 0.01 in linear space). *Hint: use <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html">```np.where```</a>*
- Print out the corresopnding words, for each group. *Hint: the ```.columns``` attribute of a dataframe gives you the column names, which should be in the same order as your feature probabilities...*


Do you observe any differences? What could be a problem here?

In [44]:
# Code here.
controls_feats, patients_feats = nb.feature_log_prob_
print("Features with high log prob under patient model:", data_s.columns[np.where(patients_feats>-4.5)])
print("Features with high log prob under control model:", data_s.columns[np.where(controls_feats>-4.5)])

Features with high log prob under patient model: Index(['de', 'der', 'sdan', 'i', 'den', 'er', 'det', 'at', 'jeg', 'og', 'ikke',
       'en', 's', 'lille', 'store', 'ud', 'var', 'trekant', 'h'],
      dtype='object')
Features with high log prob under control model: Index(['de', 'der', 'sdan', 'i', 'den', 'det', 'at', 'og', 'ikke', 'en', 's',
       'lille', 'rde', 'bl', 'var', 'trekant'],
      dtype='object')


*Take Notes here.*

## 5. Feature representation and feature selection 
There are a few things we could do to improve our model, such as tuning the smoothing parameter $\alpha$ in a cross-validation experiment (below). However, the most likely cause for failure is that we have too many features, and too little data. So let's start by working on that.
- try with binary features
- remove stopwords

## 6. Naive Bayes for Depression [A]

Now it's your turn: Can you apply the techniques you have learned to build a model for depression?

Think about the following before you start:
- Do you want to use 2 classes (depressed/not depressed) or 3 (first-time depressed/chronically depressed/not depressed)?
- How does your choice affect class imbalance? 
- How do you want to split your dataset for training and testing?
- Which dummy baseline will you use?
- Which features/feature representation seems most useful?

*Note: If you are short on time, skip to task 7 now and come back to this later, so we can wrap this up together.*

*Take some notes here:*


In [ ]:
#Code here.

## 7. Findings

Think about your findings. What have they taught you?
- What are the problems with the Naive Bayes approach?
- Do you think it is likely that the Naive Bayes assumptions were met? Why/why not?
- Have you learned anything about depression/scizophrenia?
- What's missing?
- How could this approach be used to investigate a specific research question? What additional variables/data would you need for this?

*Take some notes here:*

If you have time, you can read more about the Naive Bayes algorithm and how to improve its performance <a href="https://machinelearningmastery.com/better-naive-bayes/">here</a>.

## 8. Going Further: Finding informative features with Decision Trees [A]